## Transformations in Political Economy - Technological Change and Populism (POL63102)
### Coding Session 3: Multivariate Linear Regression

---
This document guides you through coding session 3. Please try to follow the instructions on your own PC and feel free to ask questions if something is unclear. After this session you should be able to do the following:

- Implement Multivariate Linear Regression
- Use Robust Standard Errors
---

Let's start as usual by importing modules and loading data.

In [16]:
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import os
from pathlib import Path

df = pd.read_stata('C:/Users/felix/Dropbox/HfP/Teaching/SoSe21/Populism_Course/data/Autor_data_extract.dta')

Remember that the data covers 3,722 county-district cells and contains amongst others the following variables:
- *d2_shnr_2002_Y* are the main outcome variables measuring the change in the Republican two-party vote share percentage between year 2002 and year Y = 2004, ..., 2016
- *d_imp_usch_pd* is the main independent variable for local labor market exposure to import competition from China
- *l_shind_manuf_cbp* is a control variable for manufacturing employment share

Let's produce summary statistics for these variables:

In [17]:
df[["d2_shnr_2002_2016","d_imp_usch_pd", "l_shind_manuf_cbp"]].describe().T

,count,mean,std,min,25%,50%,75%,max
d2_shnr_2002_2016,3767.0,8.358822,25.995949,-95.661407,-6.477878,6.811324,23.068745,100.000000
d_imp_usch_pd,3772.0,0.754591,0.652882,-0.259663,0.343260,0.623485,0.966137,6.079161
l_shind_manuf_cbp,3772.0,0.189763,0.097713,0.001083,0.115888,0.170197,0.257179,0.552423


In a multivariate regression model, we add more variables to the right of the regression equation:

\begin{align}
y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \epsilon
\end{align}

what can also be written in matrix notation as

\begin{align}
y = \textbf{X'}\beta + \epsilon
\end{align}

where the transposed matrix $\textbf{X'}$ is composed of all k variables $x_1, x_2, ... , x_k$ and a constant vector of 1s in the regression. 

Let's add the employment share in manufacturing to our regression model. Notice that in the formula from the *ols* function you need a "+":

In [19]:
results = smf.ols('d2_shnr_2002_2016 ~ d_imp_usch_pd + l_shind_manuf_cbp', data=df).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:      d2_shnr_2002_2016   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                  0.019
Method:                 Least Squares   F-statistic:                     37.11
Date:                Fri, 11 Jun 2021   Prob (F-statistic):           1.10e-16
Time:                        15:02:31   Log-Likelihood:                -17581.
No. Observations:                3767   AIC:                         3.517e+04
Df Residuals:                    3764   BIC:                         3.519e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             2.0166      0.91

You can also access summary of the results in table format like this:

In [20]:
results.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,2.0166,0.919,2.195,0.028,0.216,3.818
d_imp_usch_pd,-3.6655,0.848,-4.322,0.000,-5.328,-2.003
l_shind_manuf_cbp,47.9902,5.668,8.466,0.000,36.877,59.104


Note that the coefficient on import competition is $\hat \beta_1=-3.67$ (as compared to  $\hat \beta_1=1.03$ in the simple model in the previous coding session). Note that using robust standard errors does not affect coefficient size, but their standard errors and hence statistical significance.

Have you noticed the footnote saying:
*Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.*?

Yes, we should use robust standard errors if we have heteroskedasticity or even cluster standard errors. Let's see how this is being implemented.

In [23]:
(smf.ols('d2_shnr_2002_2016 ~ d_imp_usch_pd + l_shind_manuf_cbp', data=df)
    .fit(cov_type='HC1')
    .summary()
    .tables[1])

,coef,std err,z,P>|z|,[0.025,0.975]
Intercept,2.0166,0.887,2.274,0.023,0.278,3.755
d_imp_usch_pd,-3.6655,0.773,-4.740,0.000,-5.181,-2.150
l_shind_manuf_cbp,47.9902,5.430,8.838,0.000,37.348,58.632


Or alternatively, we can go back to existing results and recompute from those:

In [21]:
print(results.get_robustcov_results('HC2').summary())

                            OLS Regression Results                            
Dep. Variable:      d2_shnr_2002_2016   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                  0.019
Method:                 Least Squares   F-statistic:                     39.79
Date:                Fri, 11 Jun 2021   Prob (F-statistic):           7.97e-18
Time:                        15:08:14   Log-Likelihood:                -17581.
No. Observations:                3767   AIC:                         3.517e+04
Df Residuals:                    3764   BIC:                         3.519e+04
Df Model:                           2                                         
Covariance Type:                  HC2                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             2.0166      0.88

Note how coefficient size is unchanged, but standard errors are different. This might have implications for statistical significance.

**Exercise:** 
* What does it mean that $\hat \beta_1$ changes when including manufacturing employment share as another explanatory variable? 
* Discuss whether you might want to use clustered standard errors in this setting. 
Note: For implementation of clustered and two-way clustered standard errors, see https://aeturrell.github.io/coding-for-economists/econmt-regression.html.

---
**Congratulations! This is the end of coding session 3.**